# Part 02
본 02 파트에서 데이터 전처리를 다음과 같이 수행했습니다.
- 범주형 데이터 값들 경우 캐멀형으로 되어 있거나 소문자로 되어 있는 단어들을 통합하기 위해서 소문자로 통합했습니다.
- bottler와 age 값들의 경우, 제품명이나 제품 정보 등 다른 연관 피처에서 얻을 수 있는 정보와 도메인 정보를 활용해 최대한 null을 대체하였습니다.
- 텍스트 피쳐들의 경우, null 값은 'no description' 혹은 'no specified'와 같이 처리하였습니다.
- style 값의 경우, 이 프로젝트의 목적에서 중요한 피처로 작용하기 때문에 null을 처리하기 위해서 3가지 작업을 수행하였습니다.
    1) 지역/보틀러 기반 1차 대체
    2) KNN imputer : 다른 피처들 사용해서 유사한 위스키 5~10개 찾아 평균값 채움
    3) 텍스트 기반 보정 : characters/ nose/ palate/ finish
   
    이 세가지 값들의 결과를 서로 비교하여 먼저 값들이 큰 차이를 가지는지 점검하였고 유의미한 큰 차이가 없을 경우 텍스트 기반 보정(3번 작업 결과)를 최종값으로, 차이가 있을 경우에는 그들의 평균값들로 최종값을 결정하였습니다.



# Summary


* 전처리 과정을 통해, 데이터의 평균 결측률을 약 26.47%% → 16.38% 수준으로 감소시켰으며,

* 풍미(style) 관련 피처의 신뢰도를 향상시켜 이후 클러스터링 및 추천 시스템 설계 단계의 기반을 마련하였다.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import gc

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
%matplotlib inline

!pip install koreanize-matplotlib
import koreanize_matplotlib

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/week_hands_on_project5/whiskey/whisky_data.csv')
df.columns = df.columns.str.lower()
df.head()

,name,main_info,volume_alcohol,price,style_body,style_richness,style_smoke,style_sweetness,characters,food_paring,bottler,age,country,region,casktype,colouring,vintage,nose,palate,finish,reviews,url
0,Blended Scotch TB-BSW6 Year Old Thompson Bros,Blended Scotch Whisky,70cl / 46%,£34.25,3.0,3.0,0.0,3.0,"Figs, Sultana, Cinnamon, Nutmeg, Toffee, Coffee",NaN,NaN,6 year old,scotland,unknown_scotland,not specified,no,NaN,"Plum sauce, miso, golden sultanas, cinder toff...","Golden sultanas, raisin loaf, buttered Jamaica...","Soft warm spice; allspice, nutmeg and cinnamon...","Stunning stuff for the money, might well be my...",https://www.thewhiskyexchange.com/p/67983/blen...
1,Johnnie Walker Blue Label,Blended Scotch Whisky,70cl / 40%,£158,3.0,3.0,1.0,2.0,"Rose, Hazelnut, Honey, Smoke, Dark Chocolate, ...","Octopus, Monkfish, Fondue",NaN,NaN,scotland,unknown_scotland,not specified,yes,NaN,"Smooth, silky cereal, syrupy flapjacks, polish...",A silky caress. Unexpectedly strong spicy entr...,A perfectly-integrated slow fade of the palate...,Un elixir. || Excellent Whisky. Smooth on the ...,https://www.thewhiskyexchange.com/p/552/johnni...
2,The Big Ben,Blended Scotch Whisky,50cl / 40%,£23.50,NaN,NaN,NaN,NaN,no description,NaN,NaN,NaN,scotland,unknown_scotland,not specified,yes,NaN,"Medium weight, malty nose, with hints of orang...","Medium depth, hints of smokiness and sweetness...",Elegant finish.,Very pleased with the whiskey. Highly recommend,https://www.thewhiskyexchange.com/p/18213/the-...
3,J&B Rare,Blended Scotch Whisky,70cl / 40%,£25.25,NaN,NaN,NaN,NaN,no description,NaN,NaN,NaN,scotland,unknown_scotland,not specified,yes,NaN,no description,no description,no description,Very nice || Was delighted to find this whisky...,https://www.thewhiskyexchange.com/p/544/jb-rare
4,Dimple 15 Year OldLitre,Blended Scotch Whisky,100cl / 43%,£69.95,3.0,3.0,0.0,3.0,"Caramel, Cinnamon, Vanilla, Dark Chocolate",NaN,NaN,15 year old,scotland,unknown_scotland,not specified,NaN,NaN,no description,no description,no description,NaN,https://www.thewhiskyexchange.com/p/79846/dimp...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             1100 non-null   object 
 1   main_info        1100 non-null   object 
 2   volume_alcohol   1100 non-null   object 
 3   price            1100 non-null   object 
 4   style_body       618 non-null    float64
 5   style_richness   618 non-null    float64
 6   style_smoke      611 non-null    float64
 7   style_sweetness  618 non-null    float64
 8   characters       1100 non-null   object 
 9   food_paring      30 non-null     object 
 10  bottler          505 non-null    object 
 11  age              532 non-null    object 
 12  country          1100 non-null   object 
 13  region           1100 non-null   object 
 14  casktype         1100 non-null   object 
 15  colouring        469 non-null    object 
 16  vintage          219 non-null    float64
 17  nose          

In [ ]:
overall_missing_rate = df.isnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100
print(f"전체 평균 결측률: {overall_missing_rate:.2f}%")

전체 평균 결측률: 26.47%


### main info

In [ ]:
df['main_info'] = df['main_info'].str.lower()

In [ ]:
import re

def extract_whisky_type(text):
    if not isinstance(text, str):
        return "unknown"

    text = text.lower()

    # 주요 타입 키워드
    patterns = {
        "single malt": r"single\s+malt",
        "blended malt": r"blended\s+malt",
        "blended": r"blended(?!\s+malt)",
        "bourbon": r"bourbon",
        "pot still": r"pot\s+still",
        "grain": r"grain",
        "rye": r"rye",
        "corn": r"corn",
        "single pot still": r"single\s+pot\s+still",
    }

    for whisky_type, pat in patterns.items():
        if re.search(pat, text):
            return whisky_type

    return "other"

df["whisky_type"] = df["main_info"].apply(extract_whisky_type)

print(df["whisky_type"].value_counts())

whisky_type
single malt     449
blended         361
blended malt    142
bourbon          46
grain            32
pot still        32
other            24
rye              13
corn              1
Name: count, dtype: int64


### 0. volume/ alcohol

In [ ]:
df['volume'] = df['volume_alcohol'].str.split(' / ').str[0]
df['alcohol'] = df['volume_alcohol'].str.split(' / ').str[1]

df['volume(cl)'] = df['volume'].str.replace('cl','', regex=False).astype(float)
df['alcohol(%)'] = df['alcohol'].str.replace('%','', regex=False).astype(float)

### Price

In [ ]:
df['price(£)'] = df['price'].str.replace('£','',regex=False).str.replace(',','',regex=False).astype(float)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 28 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             1100 non-null   object 
 1   main_info        1100 non-null   object 
 2   volume_alcohol   1100 non-null   object 
 3   price            1100 non-null   object 
 4   style_body       618 non-null    float64
 5   style_richness   618 non-null    float64
 6   style_smoke      611 non-null    float64
 7   style_sweetness  618 non-null    float64
 8   characters       1100 non-null   object 
 9   food_paring      30 non-null     object 
 10  bottler          505 non-null    object 
 11  age              532 non-null    object 
 12  country          1100 non-null   object 
 13  region           1100 non-null   object 
 14  casktype         1100 non-null   object 
 15  colouring        469 non-null    object 
 16  vintage          219 non-null    float64
 17  nose          

### region / country

In [ ]:
df['region'] = df['region'].str.lower()
df['country'] = df['country'].str.lower()

### bottler

In [ ]:
df.bottler.unique()

array([nan, 'compass box', 'distillery bottling', 'berry bros & rudd',
       'watt whisky', 'the artful dodger', 'gleann mor',
       'chivas brothers', 'elixir distillers', 'the last drop',
       'signatory', 'mossburn', 'single cask nation', 'douglas laing',
       'hunter laing', 'duncan taylor', 'new riff distillery',
       'gordon & macphail', 'the whisky exchange', 'ferg and harris',
       'la maison du whisky', 'artful dodger', 'thompson bros',
       'john crabbie', 'wilson & morgan', 'hidden spirits',
       'hart brothers', 'Distillery Bottling', 'speciality drinks ltd',
       'north star spirits', 'finn thomson', 'carn mor', 'jack wiebers',
       'blackadder', 'decadent drinks', "scott's selection",
       'scotch malt whisky society', "cadenhead's", 'mcbain & charles',
       'first cask', 'suntory', 'wemyss', 'the lost distillery company',
       'adelphi', 'whisky works', 'the whisky agency', 'moon import'],
      dtype=object)

In [ ]:
import re, unicodedata

# 1. bottler 소문자화, 유니코드 정규화, ' 통일, 다중 공백 제거
def norm_text(s: str) -> str:

    s = "" if pd.isna(s) else str(s)
    s = s.lower()
    s = unicodedata.normalize("NFKD", s)
    s = re.sub(r"[’‘´`']", "'", s)   # 다양한 따옴표를 ' 로 통일
    s = re.sub(r"\s+", " ", s).strip()
    return s
# 이름 정규화 칼럼
df["name_norm"]    = df["name"].map(norm_text)
# 보틀러 정규화 칼럼
df["bottler_norm"] = df["bottler"].map(norm_text)

# ----------------------------------------------------------------

# 2. 동일 보틀러가 여러 표기로 들어온 경우
ALIAS_TO_CANON = {
    "distillery bottling": "distillery bottling",
    "the artful dodger": "artful dodger",
    "artful dodger": "artful dodger",

}

# 동일 보틀러 이름 표기 다른 경우 replace 처리
df["bottler_canon"] = df["bottler_norm"].replace(ALIAS_TO_CANON)

#----------------------------------------------------------------
# 3. 이름 속 보틀러 패턴 regex, label dict
IB_PATTERNS = {
    r"\bthompson\s*bros?\b": "thompson bros",
    r"\bcompass\s*box\b": "compass box",
    r"\bwoven\b": "woven",
    r"\bjames\s*eadie\b": "james eadie",
    r"\bberry\s*bros?\.?\s*&\s*rudd\b": "berry bros & rudd",
    r"\bwatt\s*whisky\b": "watt whisky",
    r"\bartful\s*dodger\b": "artful dodger",
    r"\bgleann\s*mor\b": "gleann mor",
    r"\bpure\s*scot\b": "pure scot",
    r"\bcoachbuilt\b": "coachbuilt",
    r"\belixir\s*distillers\b": "elixir distillers",
    r"\bthe\s*last\s*drop\b": "the last drop",
    r"\bwhisky\s*works\b": "whisky works",
    r"\bsignatory\b": "signatory",
    r"\bmossburn\b": "mossburn",
    r"\bsingle\s*cask\s*nation\b": "single cask nation",
    r"\bdouglas\s*laing\b": "douglas laing",
    r"\bhunter\s*laing\b": "hunter laing",
    r"\bduncan\s*taylor\b": "duncan taylor",
    r"\bgordon\s*&\s*macphail\b": "gordon & macphail",
    r"\bthe\s*whisky\s*exchange\b": "the whisky exchange",
    r"\bferg\s*and\s*harris\b": "ferg and harris",
    r"\bla\s*maison\s*du\s*whisky\b": "la maison du whisky",
    r"\bjohn\s*crabbie\b": "john crabbie",
    r"\bwilson\s*&\s*morgan\b": "wilson & morgan",
    r"\bhidden\s*spirits\b": "hidden spirits",
    r"\brest\s*&\s*be\s*thankful\b": "rest & be thankful",
    r"\bhart\s*brothers\b": "hart brothers",
    r"\bspeciality\s*drinks\s*ltd\b": "speciality drinks ltd",
    r"\bnorth\s*star\s*spirits\b": "north star spirits",
    r"\bfinn\s*thomson\b": "finn thomson",
    r"\bcarn\s*mor\b": "carn mor",
    r"\bjack\s*wiebers\b": "jack wiebers",
    r"\bblackadder\b": "blackadder",
    r"\bdecadent\s*drinks\b": "decadent drinks",
    r"\bscott'?s\s*selection\b": "scott's selection",
    r"\bscotch\s*malt\s*whisky\s*society\b|\bsmws\b": "scotch malt whisky society",
    r"\bcadenhead'?s\b": "cadenhead's",
    r"\bmcbain\s*&\s*charles\b": "mcbain & charles",
    r"\bfirst\s*cask\b": "first cask",
    r"\bold\s*perth\b": "old perth",
    r"\bwemyss\b": "wemyss",
    r"\bthe\s*lost\s*distillery\s*company\b": "the lost distillery company",
    r"\bgladstone\s*axe\b": "gladstone axe",
    r"\badelphi\b": "adelphi",
    r"\bangels?'?\s*nectar\b": "angels' nectar",
    r"\bthe\s*whisky\s*agency\b": "the whisky agency",
    r"\bmoon\s*import\b": "moon import",
    r"\bsheep\s*dip\b": "sheep dip",
    r"\btwo\s*stacks\b": "two stacks",
    r"\bjj\s*corry\b|\bj\s*&?\s*j\s*corry\b": "jj corry",
    #OB
    r"\bchivas\s*brothers\b": "chivas brothers",
    r"\bsuntory\b": "suntory",
    r"\bnew\s*riff\s*distillery\b": "new riff distillery",
}
# re.compile
IB_REGEX = [(re.compile(regex), label) for regex, label in IB_PATTERNS.items()]

#4.  보틀러가 null 인 값에 대해서 이름에서 먼저 추론하도록 함
def infer_bottler_from_name(name_norm: str) -> str:
    for rx, lab in IB_REGEX:
        if rx.search(name_norm):
            return lab
    return None

mask_na = df["bottler_canon"].isna() | (df["bottler_canon"] == "")
df.loc[mask_na, "bottler_canon"] = df.loc[mask_na, "name_norm"].map(infer_bottler_from_name)


# 5. 그래도 못 찾은 경우 : OB 기본값
df["bottler_canon"] = df["bottler_canon"].fillna("distillery bottling")



# ---------------------------------------------------------------------------
OB_WHITELIST = {
    "distillery bottling",
    "chivas brothers",
    "suntory",
    "new riff distillery",
}

# 6.기본은 IB(=1), 화이트리스트면 OB(=0)
df["is_independent"] = (~df["bottler_canon"].isin(OB_WHITELIST)).astype("int8")

print(df["bottler_canon"].value_counts().head(20))
print(df["is_independent"].value_counts(dropna=False))


bottler_canon
distillery bottling           814
signatory                      37
douglas laing                  31
compass box                    27
gordon & macphail              22
elixir distillers              15
berry bros & rudd              13
the whisky exchange            13
watt whisky                     9
woven                           9
old perth                       8
chivas brothers                 7
mossburn                        6
two stacks                      6
thompson bros                   5
single cask nation              5
scotch malt whisky society      5
artful dodger                   5
duncan taylor                   4
decadent drinks                 4
Name: count, dtype: int64
is_independent
0    825
1    275
Name: count, dtype: int64


In [ ]:
display(df[df['is_independent']==1]['bottler_canon'].value_counts().head(10))
display(df[df['is_independent']==0]['bottler_canon'].value_counts())

,count
bottler_canon,
signatory,37
douglas laing,31
compass box,27
gordon & macphail,22
elixir distillers,15
berry bros & rudd,13
the whisky exchange,13
watt whisky,9
woven,9


,count
bottler_canon,
distillery bottling,814
chivas brothers,7
suntory,3
new riff distillery,1


In [ ]:
df[['name','age','vintage']].sample(20)

,name,age,vintage
837,Yoichi 200020 Year Old Anniversary Blend ltd E...,20 year old,2000.0
561,Macallan The Harmony Collection Vibrant Oak,NaN,NaN
600,Benriach The Sixteen16 Year Old,16 year old,NaN
314,Girvan 200219 Year Old Old Particular,NaN,2002.0
706,Aberlour 196921 Year Old Bot.1991 Sherry Cask,21 year old,1969.0
267,White Heather 8 Year OldDe Luxe Bot.1970s,8 year old,NaN
924,Compass Box The Peat MonsterGlass Set,NaN,NaN
614,Benriach 199131 Year Old Berry Bros & Rudd,31 year old,1991.0
772,Yamazaki 18 Year Old Mizunara100th Anniversary,18 year old,NaN
1061,Dingle Lunasa Single Malt,NaN,NaN


In [ ]:
import re


# 0) 전처리: 소문자 + 자주 붙는 토큰 분리 + 기호 정리
def norm_name_lower(s: str) -> str:
    if not isinstance(s, str):
        return ""
    x = s.lower().strip()

    # 유니코드 dash 통합
    x = re.sub(r"[–—−‐]", "-", x)

    # 글자+연도 붙음 → 분리 (예: karuizawa1995 → karuizawa 1995)
    x = re.sub(r"([a-z])(\d{4})\b", r"\1 \2", x)

    # 슬래시/대시로 연-연 표기 '1998/2013' '1998-2013' 앞뒤 공백 보강
    x = re.sub(r"(\d{4})\s*[/\-]\s*(\d{4})", r"\1 - \2", x)

    # bot/btl/btld/bt 붙어쓰기 끊기 (예: oldbot. -> old bot.)
    x = re.sub(r"(old)(bot\.)", r"\1 \2", x)
    x = re.sub(r"(case)(bot\.)", r"\1 \2", x)

    # 다양한 축약형 표기 앞뒤 공백 보강
    x = re.sub(r"([a-z])\s*(bot\.|btl\.|btld\.|bt\.|b\.)\b", r"\1 \2", x)
    x = re.sub(r"([a-z])\s*(distilled|dist\.|dstl\.|d\.)\b", r"\1 \2", x)
    x = re.sub(r"([a-z])\s*(vintage|vtg\.)\b", r"\1 \2", x)

    # cask 표기 다양성 처리 (caskno., cask#, cask no., barrel, puncheon 등)
    x = re.sub(r"(cask)(no\.|#)", r"\1 \2", x)
    x = re.sub(r"(cask)(\s*[a-z0-9])", r"\1 \2", x)  # caskCU70095 → cask CU70095
    x = re.sub(r"\b(barrel|puncheon|hogshead|butt)\b", "cask", x)  # 통상 캐스크로 통일

    # 'bot.' 오타/ocr 교정 (botl., bttl., boto. 등 흔한 오인식 몇 개)
    x = re.sub(r"\bbotl?\.?\b", "bot.", x)
    x = re.sub(r"\bbttl?\.?\b", "bot.", x)

    # 4자리 연도 + 바로 뒤에 age 표현 붙은 경우 (예: 199515 year old → 1995 15 year old)
    x = re.sub(r"(\d{4})(\d{1,2})\s*(year|yo|yrs?)\b", r"\1 \2 \3", x)
    x = re.sub(r"(\d{4})(\d{1,2})(yo|yrs?|y\.?o\.?)\b", r"\1 \2 \3", x)
    x = re.sub(r"(\d{4})(\d{1,2})\.(yo|yrs?|y\.?o\.?)\b", r"\1 \2 \3", x)

    # 콤마 뒤 공백
    x = re.sub(r",(?!\s)", ", ", x)

    # 다중 공백 축소
    x = re.sub(r"\s+", " ", x).strip()
    return x

# 1) 패턴(모두 소문자 기준)
RE_BOT_YEAR    = re.compile(r"\b(bottled|bot\.|btl\.|btld\.|bt\.|b\.)\s*(?:in\s*)?(18\d{2}|19\d{2}|20\d{2})\b")
RE_BOT_DECADE  = re.compile(r"\b(bottled|bot\.|btl\.|btld\.|bt\.|b\.)\s*(18\d0s|19\d0s|20\d0s)\b")

# '12 year old', '12 yo', '3 yrs' 등 → 반드시 숙성 의미 단어 동반
RE_AGE_TEXT    = re.compile(r"\b(\d{1,2})\s*(?:year\s*old|years?|yrs?|yo|y\.?o\.?)\b")

# vintage/증류 연도 (명시적)
RE_VINTAGE     = re.compile(r"\b(vintage|vtg\.|distilled|dist\.|dstl\.|d\.)\s*(?:in\s*)?(18\d{2}|19\d{2}|20\d{2})\b")

# 독립 4자리 연도(보수적 사용)
RE_YEAR_ALONE  = re.compile(r"\b(18\d{2}|19\d{2}|20\d{2})\b")

# 연도-연도 범위 (정규화 단계에서 '1998 - 2013' 으로 바꿨으므로 그대로 캡처)
RE_YEAR_RANGE  = re.compile(r"\b(18\d{2}|19\d{2}|20\d{2})\s*-\s*(18\d{2}|19\d{2}|20\d{2})\b")

# 캐스크 코드(알파넘/하이픈/슬래시/샤프 허용)
RE_CASK        = re.compile(r"\bcask\s*([a-z0-9][a-z0-9\-/#]*)\b")

def parse_from_name(name_raw: str):
    """
    반환:
      bottling_year: Int (정확 연도만)
      bottling_decade: '1970s' 같은 문자열
      age_years_text: '12 year old' 같은 원문형
      age_years: Int (가능할 때 추론)
      vintage_maybe: Int 또는 '1970s'
      cask_code: str (있으면)
    """
    out = {
        "bottling_year": None,
        "bottling_decade": None,
        "age_years_text": None,
        "age_years": None,
        "vintage_maybe": None,
        "cask_code": None,
    }
    s = norm_name_lower(name_raw)

    # (0) 연도-연도 범위가 단독으로 있으면, 휴리스틱으로 vintage~bottling로 추정
    # 예: 'yamazaki 1998/2013' -> 1998 vintage, 2013 bottled
    m_range = RE_YEAR_RANGE.search(s)
    if m_range:
        y1, y2 = int(m_range.group(1)), int(m_range.group(2))
        if 1800 <= y1 <= 2030 and 1800 <= y2 <= 2030 and y1 < y2:
            out["vintage_maybe"], out["bottling_year"] = y1, y2

    # (A) bottling year/decade
    m = RE_BOT_YEAR.search(s)
    if m:
        y = int(m.group(2))
        if 1800 <= y <= 2030:
            out["bottling_year"] = y
    else:
        m = RE_BOT_DECADE.search(s)
        if m:
            out["bottling_decade"] = m.group(2)  # 예: '1990s'

    # (B) age 텍스트 (숫자만 저장 금지 → 의미 포함 텍스트로)
    m = RE_AGE_TEXT.search(s)
    if m:
        yrs = int(m.group(1))
        out["age_years_text"] = f"{yrs} year old"
        out["age_years"] = yrs

    # (C) vintage 명시적
    m = RE_VINTAGE.search(s)
    if m:
        y = int(m.group(2))
        if 1800 <= y <= 2030:
            out["vintage_maybe"] = y

    # (C2) 캐스크 코드
    m = RE_CASK.search(s)
    if m:
        out["cask_code"] = m.group(1).upper()

    # (D) 보수적 휴리스틱:
    #   - 'bot' 근처의 연도는 bottling 후보, 그 외 연도는 vintage 후보
    if out["vintage_maybe"] is None:
        years = [(int(m2.group(1)), m2.span()) for m2 in RE_YEAR_ALONE.finditer(s)]
        def near_bot(span):
            i, j = span
            window = s[max(0, i-10):min(len(s), j+10)]
            return bool(re.search(r"\b(bottled|bot\.|btl\.|btld\.|bt\.|b\.)\b", window))

        if years:
            # 두 연도 이상 & bottling이 비어있으면 bot 근접/비근접 휴리스틱
            if len(years) >= 2 and out["bottling_year"] is None:
                near = [(y, sp) for y, sp in years if near_bot(sp)]
                far  = [(y, sp) for y, sp in years if not near_bot(sp)]
                if len(near) == 1 and len(far) >= 1:
                    out["bottling_year"] = near[0][0]
                    # bottling과 다른 연도 중 가장 이른 연도를 vintage로
                    out["vintage_maybe"] = min(y for y, _ in far if y != out["bottling_year"])

            # 이미 bottling이 있으면, 다른 연도 중 가장 이른 것을 vintage 후보로
            if out["vintage_maybe"] is None and out["bottling_year"] is not None:
                candidates = [y for y, _ in years if y != out["bottling_year"]]
                if candidates:
                    out["vintage_maybe"] = min(candidates)

    return out

# 2) DataFrame에 적용 (df에 'name' 컬럼 존재 가정)
# parsed 결과를 펼쳐서 병합
parsed = df["name"].apply(parse_from_name).apply(pd.Series)

for col in ["bottling_year", "bottling_decade", "age_years_text", "age_years", "vintage_maybe", "cask_code"]:
    if col not in df.columns:
        df[col] = parsed[col]
    else:
        df[col] = df[col].where(df[col].notna(), parsed[col])

# (선택) 정확 연도는 nullable 정수로 캐스팅
for col in ["bottling_year", "age_years", "vintage_maybe"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int16")

### 5. age / vintage

In [ ]:
df_age_na = df[df['age'].isna()]
df_vintage_na = df[df['vintage'].isna()]
df_age_vintage_na = df[df['age'].isna() & df['vintage'].isna()]

In [ ]:
# 끝에 결측 플래그 추가 # is_age_missing, is_vintage_missing

In [ ]:
# name에서 찾은 age 데이터 입력
mask_a_f = df['age'].isna() & df['age_years_text'].notna()
if mask_a_f.any():
  index_year_text_t = mask_a_f.index
  df.loc[index_year_text_t, 'age'] = df.loc[index_year_text_t, 'age_years_text']
else :
  print("해당 행 없음")

# name에서 찾은 vintage 데이터
mask_v_f = df['vintage'].isna() & df['vintage_maybe'].notna()
if mask_v_f.any():
  index_v_f = mask_v_f.index
  df.loc[index_v_f, 'vintage'] = df.loc[index_v_f, 'vintage_maybe']
else :
  print("해당 행 없음")

In [ ]:
# bottler year 를 찾은 행에 대해서 age + vintage 모두 값이 있는 행 제외하고
df_bottler_year_true = df[
    (~df['bottling_year'].isna()) &
    (~df['age'].isna()) & (~df['vintage'].isna())
]
# 1. age가 있지만 vintage 가 없는 행 vintage 값 계산
mask_age_true = (
    df_bottler_year_true['age'].notna() &
    df_bottler_year_true['vintage'].isna()
)

if mask_age_true.any():
    index_age_t = df_bottler_year_true[mask_age_true].index
    df.loc[index_age_t, 'vintage'] = (
        df.loc[index_age_t, 'bottling_year'] - df.loc[index_age_t, 'age']
    )
# 2. age가 없지만 vintage 가 있는 행 age 값 계산

mask_vintage_true = (
    df_bottler_year_true['age'].isna() &
    df_bottler_year_true['vintage'].notna()
)
if mask_vintage_true.any():
  index_vintage_t = df_bottler_year_true[mask_vintage_true].index
  df.loc[index_vintage_t, 'age'] = (
      df.loc[index_vintage_t, 'bottling_year'] - df.loc[index_vintage_t, 'vintage']
  )

In [ ]:
df.head()

,name,main_info,volume_alcohol,price,style_body,style_richness,style_smoke,style_sweetness,characters,food_paring,bottler,age,country,region,casktype,colouring,vintage,nose,palate,finish,reviews,url,whisky_type,volume,alcohol,volume(cl),alcohol(%),price(£),name_norm,bottler_norm,bottler_canon,is_independent,bottling_year,bottling_decade,age_years_text,age_years,vintage_maybe,cask_code
0,Blended Scotch TB-BSW6 Year Old Thompson Bros,blended scotch whisky,70cl / 46%,£34.25,3.0,3.0,0.0,3.0,"Figs, Sultana, Cinnamon, Nutmeg, Toffee, Coffee",NaN,NaN,None,scotland,unknown_scotland,not specified,no,<NA>,"Plum sauce, miso, golden sultanas, cinder toff...","Golden sultanas, raisin loaf, buttered Jamaica...","Soft warm spice; allspice, nutmeg and cinnamon...","Stunning stuff for the money, might well be my...",https://www.thewhiskyexchange.com/p/67983/blen...,blended,70cl,46%,70.0,46.0,34.25,blended scotch tb-bsw6 year old thompson bros,,thompson bros,1,<NA>,None,None,<NA>,<NA>,None
1,Johnnie Walker Blue Label,blended scotch whisky,70cl / 40%,£158,3.0,3.0,1.0,2.0,"Rose, Hazelnut, Honey, Smoke, Dark Chocolate, ...","Octopus, Monkfish, Fondue",NaN,None,scotland,unknown_scotland,not specified,yes,<NA>,"Smooth, silky cereal, syrupy flapjacks, polish...",A silky caress. Unexpectedly strong spicy entr...,A perfectly-integrated slow fade of the palate...,Un elixir. || Excellent Whisky. Smooth on the ...,https://www.thewhiskyexchange.com/p/552/johnni...,blended,70cl,40%,70.0,40.0,158.00,johnnie walker blue label,,distillery bottling,0,<NA>,None,None,<NA>,<NA>,None
2,The Big Ben,blended scotch whisky,50cl / 40%,£23.50,NaN,NaN,NaN,NaN,no description,NaN,NaN,None,scotland,unknown_scotland,not specified,yes,<NA>,"Medium weight, malty nose, with hints of orang...","Medium depth, hints of smokiness and sweetness...",Elegant finish.,Very pleased with the whiskey. Highly recommend,https://www.thewhiskyexchange.com/p/18213/the-...,blended,50cl,40%,50.0,40.0,23.50,the big ben,,distillery bottling,0,<NA>,None,None,<NA>,<NA>,None
3,J&B Rare,blended scotch whisky,70cl / 40%,£25.25,NaN,NaN,NaN,NaN,no description,NaN,NaN,None,scotland,unknown_scotland,not specified,yes,<NA>,no description,no description,no description,Very nice || Was delighted to find this whisky...,https://www.thewhiskyexchange.com/p/544/jb-rare,blended,70cl,40%,70.0,40.0,25.25,j&b rare,,distillery bottling,0,<NA>,None,None,<NA>,<NA>,None
4,Dimple 15 Year OldLitre,blended scotch whisky,100cl / 43%,£69.95,3.0,3.0,0.0,3.0,"Caramel, Cinnamon, Vanilla, Dark Chocolate",NaN,NaN,15 year old,scotland,unknown_scotland,not specified,NaN,<NA>,no description,no description,no description,NaN,https://www.thewhiskyexchange.com/p/79846/dimp...,blended,100cl,43%,100.0,43.0,69.95,dimple 15 year oldlitre,,distillery bottling,0,<NA>,None,15 year old,15,<NA>,None


In [ ]:
df['age_'] = (
    df['age']
    .str.extract(r'(\d+)')
    .astype(float)
)
df['age_int'] = (
    df['age']
    .str.extract(r'(\d+)')
    .astype(float)
)
df['age_int'] = df['age_int'].fillna(df.groupby('region')['age_int'].transform('mean'))
df['is_NAS'] = df['age'].isna().astype(int)

df['age_int'] = df['age_int'].fillna(df.groupby('country')['age_int'].transform('mean'))

In [ ]:
df["vintage_clean"] = (
    df["vintage"]
    .astype(str)
    .str.extract(r"(\d{4})")
    .astype(float)
    .astype("Int64")
)
df["bottling_decade_clean"] = (
    df["bottling_decade"]
    .astype(str)
    .str.rstrip("s")
    .str.extract(r"(\d{4})")
    .astype(float)
    .astype("Int64")
)

### casktype

In [ ]:
df.casktype.unique()

array(['not specified', 'sherry', 'bourbon', 'oloroso sherry finish',
       'finish', 'sherry finish', 'mizunara', 'oloroso sherry  finish',
       'pedro ximenez sherry finish', 'bourbon hogshead', 'new oak',
       'rum  finish', 'bourbon finish', 'pedro ximenez sherry  finish',
       'first fill bourbon barrel', 'oloroso hogshead finish',
       'first-fill bourbon barrel', 'refill hogshead', 'refill barrel',
       'bourbon barrel', 'refill oloroso sherry butt',
       'oloroso sherry butt', 'refill bourbon barrel', 'port finish',
       'rum finish', 'wine finish', 'first-fill barrel',
       'first-fill sherry butt', 'rum', 'oloroso sherry',
       'first fill oloroso sherry butt', 'first-fill sherry butt finish',
       'first-fill', 'first fill oloroso sherry hogshead',
       'bourbon barrel finish', 'refill sherry butt', 'first fill sherry',
       'first-fill oloroso sherry butt', 'sherry butt finish',
       'madeira finish', 'first fill oloroso sherry',
       'first-fil

In [ ]:
def simplify_cask(x):
    if pd.isna(x):
        return "unknown"
    x = x.lower().strip()
    if any(w in x for w in ["sherry", "oloroso", "pedro ximenez", "px", "amontillado", "manzanilla", "fino", "cream"]):
        return "sherry"
    elif any(w in x for w in ["bourbon", "hogshead", "new oak", "barrel", "butt"]):
        return "bourbon"
    elif any(w in x for w in ["wine", "port", "marsala", "barolo", "madeira", "amarone", "valpolicella", "bolgheri", "bordeaux", "tokaji", "moscatel"]):
        return "wine_port"
    elif "rum" in x:
        return "rum"
    elif any(w in x for w in ["cognac", "armagnac", "calvados", "brandy", "marnier"]):
        return "brandy_cognac"
    elif any(w in x for w in ["mizunara", "sakura", "cedarwood", "cherry wood", "bog oak", "hickory", "chestnut", "oak"]):
        return "mizunara_asian_wood"
    elif any(w in x for w in ["beer", "liqueur"]):
        return "other_experimental"
    else:
        return "unknown"

df["cask_group"] = df["casktype"].apply(simplify_cask)

In [ ]:
# null 값 처리
# 1. age

# 2. vintage

# 3. bottling

# 4. decade



### 8. colouring

In [ ]:
df['colouring'].value_counts()

df['colouring'] = df['colouring'].str.lower()
colouring_map = {'no':False,
                 'yes': True}
df['colouring'] = df['colouring'].map(colouring_map)

df['colouring'] = df['colouring'].fillna('Unknown')

### 9. nose/palate/finish

In [ ]:
df['nose'] = df['nose'].fillna('no description')
df['palate'] = df['palate'].fillna('no description')
df['finish'] = df['finish'].fillna('no description')

### 1. style


<계층적 보정 방식으로 null 값을 처리했습니다>
1. 지역/보틀러 기반 1차 대체
2. KNN imputer : 다른 피처들 사용해서 유사한 위스키 5~10개 찾아 평균값 채움
3. 텍스트 기반 보정 : characters/ nose/ palate/ finish

In [ ]:
# style 3가지 값은 있고 smoke 에 대한 값만 빠진 데이터 7행 0.0으로 처리
smoke_0_mask = df['style_body'].notna() & df['style_smoke'].isna()

df.loc[smoke_0_mask, 'style_smoke'] = df.loc[smoke_0_mask, 'style_smoke'].fillna(0.0)

In [ ]:
df_raw = df.copy(deep=True)

In [ ]:
df_l1 = df.copy()

In [ ]:
# ------------------------------------ 1. 지역 / 메인인포 / 보틀러 기반 1차 대체 ------------------------------------ #

style_cols = ["style_body", "style_richness", "style_smoke", "style_sweetness"]

# 1 . 데이터 타입 변경
for c in style_cols:
    df_l1[c] = pd.to_numeric(df_l1[c], errors="coerce")

def group_stats(df_l1, by):
    g = df_l1.groupby(by, dropna=False)[style_cols]
    stats = g.agg(['mean','count'])
    stats.columns = [f"{c}_{m}" for c, m in stats.columns]
    return stats

# 0) 전체 평균
global_mean = {c: df_l1[c].mean() for c in style_cols}

# 1) 지역별 평균
L1 = group_stats(df_l1, ["region"])

# 2) 지역 + 보틀러 평균
L2 = group_stats(df_l1, ["region","bottler_canon"]).reset_index()
L2 = L2.merge(L1.reset_index(), on="region", how="left", suffixes=("","__parent"))

k2 = 15  # 수축 강도
for c in style_cols:
    m, n = f"{c}_mean", f"{c}_count"
    pm = f"{c}_mean__parent"
    L2[pm] = L2[pm].fillna(global_mean[c])
    L2[f"{c}_blend"] = (L2[n]*L2[m] + k2*L2[pm]) / (L2[n] + k2)

# 3) 지역 + 메인인포 + 보틀러 평균
L3 = group_stats(df_l1, ["region","main_info","bottler_canon"]).reset_index()

# L2 blend → parent로 rename
L2_parent = L2.copy()
for c in style_cols:
    L2_parent.rename(columns={f"{c}_blend": f"{c}_parent"}, inplace=True)

L3 = L3.merge(
    L2_parent[["region","bottler_canon"] + [f"{c}_parent" for c in style_cols]],
    on=["region","bottler_canon"], how="left"
)

k3 = 20
for c in style_cols:
    m, n = f"{c}_mean", f"{c}_count"
    pm = f"{c}_parent"

    # region 평균과 매핑
    region_means = L1.reset_index()[["region", f"{c}_mean"]]
    region_means = region_means.rename(columns={f"{c}_mean": f"{c}_region_parent"})

    # merge로 붙이기
    L3 = L3.merge(region_means, on="region", how="left")

    # parent NaN → region 평균 → global
    L3[pm] = L3[pm].fillna(L3[f"{c}_region_parent"])
    L3[pm] = L3[pm].fillna(global_mean[c])

    # blended 값 계산
    L3[f"{c}_blend"] = (L3[n]*L3[m] + k3*L3[pm]) / (L3[n] + k3)

    # region_parent 컬럼 정리
    L3.drop(columns=[f"{c}_region_parent"], inplace=True)

# 채우기 단계 ------------------------------------

key3 = ["region","main_info","bottler_canon"]
key2 = ["region","bottler_canon"]

# 3단계 우선 채움
df_l1 = df_l1.merge(
    L3[key3 + [f"{c}_blend" for c in style_cols]],
    on=key3, how="left"
)
for c in style_cols:
    df_l1[c] = df_l1[c].fillna(df_l1[f"{c}_blend"])
df_l1.drop(columns=[c for c in df_l1.columns if c.endswith("_blend")], inplace=True)

# 2단계 채움
df_l1 = df_l1.merge(
    L2[key2 + [f"{c}_blend" for c in style_cols]],
    on=key2, how="left"
)
for c in style_cols:
    df_l1[c] = df_l1[c].fillna(df_l1[f"{c}_blend"])
df_l1.drop(columns=[c for c in df_l1.columns if c.endswith("_blend")], inplace=True)

# 1단계(region) 채움 → 전역 평균
for c in style_cols:
    region_means = L1.reset_index()[["region", f"{c}_mean"]]
    df_l1 = df_l1.merge(region_means, on="region", how="left")
    df_l1[c] = df_l1[c].fillna(df_l1[f"{c}_mean"])
    df_l1.drop(columns=[f"{c}_mean"], inplace=True)
    df_l1[c] = df_l1[c].fillna(global_mean[c])

# 반올림
df_l1[style_cols] = df_l1[style_cols].round(1)

In [ ]:
df_l1_out = df_l1[style_cols].copy()
df_l1_out.columns = [f"{c}_l1" for c in style_cols]
df = pd.concat([df,df_l1_out], axis=1)

In [ ]:
# ------------------------------------KNN imputer : 다른 피처들 사용해서 유사한 위스키 5~10개 찾아 평균값 채움 ------------------------------------ #
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler


num_cols = ["alcohol(%)", "price(£)", "age_int", "is_NAS"]
cat_cols = ["region", "cask_group"]

for col in ["alcohol(%)", "price(£)", "age_int", "is_NAS"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

preproc = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
    ],
    remainder="drop"
)

# 피처 행렬(거리 계산용)
X_feat = preproc.fit_transform(df[num_cols + cat_cols])

# 스타일은 거리에 미사용  NaN 채널로 붙여 KNNImputer에게만 맡김
styles_nan = np.full((len(df), len(style_cols)), np.nan, dtype=float)
M = np.concatenate([X_feat, styles_nan], axis=1)

# KNN 보간
imputer = KNNImputer(n_neighbors=7, weights="distance")
M_imp = imputer.fit_transform(M)

# 뒤의 4개 컬럼이 보간된 style 예측치
knn_pred = pd.DataFrame(
    M_imp[:, -len(style_cols):],
    columns=[f"{c}_knn" for c in style_cols],
    index=df.index
)

df = pd.concat([df, knn_pred], axis=1)
df = df.loc[:, ~df.columns.duplicated(keep="last")]
# 2) 항상 Series로 안전하게 꺼내는 헬퍼
def get_series_safe(frame: pd.DataFrame, col: str, index=None) -> pd.Series:
    if col not in frame.columns:
        # 없는 경우 NaN 시리즈 반환
        return pd.Series(np.nan, index=index if index is not None else frame.index)
    obj = frame.loc[:, col]
    if isinstance(obj, pd.DataFrame):
        # 중복 컬럼명이면 첫 번째 컬럼만 사용
        obj = obj.iloc[:, 0]
    return pd.to_numeric(obj, errors="coerce")

# ----- 최종 결합: original > stage1(_s1) > knn -----
for c in style_cols:
    s_orig   = get_series_safe(df_raw, c, index=df.index)
    s_stage1 = get_series_safe(df, f"{c}_s1", index=df.index)   # <- 접미사 _s1 일치
    s_knn    = get_series_safe(df, f"{c}_knn", index=df.index)

    df[f"{c}_final"] = (
        s_orig.combine_first(s_stage1).combine_first(s_knn)
    ).round(1)

    df[f"{c}_source"] = np.select(
        [
            s_orig.notna(),
            s_orig.isna() & s_stage1.notna(),
            s_orig.isna() & s_stage1.isna() & s_knn.notna()
        ],
        ["original", "stage1", "knn"],
        default="missing"
    )

print("Remaining NA (final):",
      df[[f"{c}_final" for c in style_cols]].isna().mean().round(3).to_dict())
for c in style_cols:
    print(c, "->", df[f"{c}_source"].value_counts(dropna=False).to_dict())

Remaining NA (final): {'style_body_final': 0.0, 'style_richness_final': 0.0, 'style_smoke_final': 0.0, 'style_sweetness_final': 0.0}
style_body -> {'original': 618, 'knn': 482}
style_richness -> {'original': 618, 'knn': 482}
style_smoke -> {'original': 618, 'knn': 482}
style_sweetness -> {'original': 618, 'knn': 482}


In [ ]:
# 점검 : 원본 데이터를 변경했는지 안했는지 확인
source_cols = [
    "style_body_source",
    "style_richness_source",
    "style_smoke_source",
    "style_sweetness_source"
]

# 행 단위로 값 확인
df["all_sources_equal"] = df[source_cols].nunique(axis=1) == 1

# 결과 확인
print(df["all_sources_equal"].value_counts())

all_sources_equal
True    1100
Name: count, dtype: int64


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline

# -------------------- 설정 --------------------
style_cols = ["style_body", "style_richness", "style_smoke", "style_sweetness"]
_text_candidates = ["characters", "food_paring", "nose", "palate", "finish", "reviews"]
text_cols = [c for c in _text_candidates if c in df.columns]
assert len(text_cols) > 0, "텍스트 컬럼이 없습니다."

MIN_TRAIN_ROWS = 20
ALPHAS = np.logspace(-3, 2, 10)
VEC_MAX_FEATURES = 2000
VEC_NGRAM = (1, 2)
VEC_MIN_DF = 2
TOP_K_TERMS = 25  # 타깃별 상위/하위 n-gram 개수

# -------------------- 텍스트 합치기 및 불용어 처리 ----------------------------
import re
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS

# 1) _text_all 정규화 함수 (원본 텍스트는 건드리지 않음)
def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.lower()
    # 흔한 표기 통일
    s = s.replace("whiskey", "whisky").replace("smokey", "smoky").replace("full bodied", "full-bodied")
    # 숫자/기호 제거 (PX, 12yo 같은 건 남기고 싶다면 정규식 조절)
    s = re.sub(r"[^a-z\s\-]", " ", s)              # 영문/하이픈만 남김
    s = re.sub(r"\b[a-z]\b", " ", s)               # 1글자 토큰 제거
    s = re.sub(r"\s{2,}", " ", s).strip()
    return s

# _text_all 만들 때 정규화 적용
_text_candidates = ["characters","food_paring","nose","palate","finish","reviews"]
text_cols = [c for c in _text_candidates if c in df.columns]
df["_text_all"] = df[text_cols].fillna("").agg(" ".join, axis=1).map(normalize_text)


# (1) 커스텀 불용어를 list로 변환
domain_stop = {
    "and","with","the","this","that","from","into","over","under","after","before",
    "very","quite","really","nice","good",
    "note","notes","hint","hints","aroma","aromas","nose","palate","finish","taste","flavour","flavor",
    "whisky","scotch","single","malt","blend","bottled","bottle","aged","years","year",
    "old","jim","jack","drank","no","wow","ice","paying","worth","box","description","yes","label","bottle",
    "attention","list"
}
custom_stopwords = list(ENGLISH_STOP_WORDS.union(domain_stop))  # <-- 핵심 수정

# (2) 이 stopwords를 쓰는 vectorizer를 파이프라인에 그대로 사용
vectorizer = TfidfVectorizer(
    stop_words=custom_stopwords,     # 이제 list
    max_features=2000,
    ngram_range=(1,2),
    min_df=2,
    max_df=0.8,
    token_pattern=r"(?u)\b[a-z][a-z\-]{2,}\b",
    sublinear_tf=True
)
#--------------------------------------------------------------------------------

has_text = df["_text_all"].str.strip().astype(bool)

# 결과 컬럼 초기화
for col in style_cols:
    final_col = f"{col}_text_final"
    src_col   = f"{col}_text_source"
    df[final_col] = pd.to_numeric(df.get(col, np.nan), errors="coerce")
    df[src_col]   = np.where(df[col].notna(), "original", "missing")

# -------------------- 학습/예측 + 모델 보관 --------------------
from sklearn.pipeline import Pipeline
from sklearn.linear_model import RidgeCV

text_models = {}

for col in style_cols:
    final_col = f"{col}_text_final"
    src_col   = f"{col}_text_source"

    y = pd.to_numeric(df[col], errors="coerce")
    mask_train = y.notna() & has_text
    mask_pred  = y.isna()  & has_text

    if mask_train.sum() < MIN_TRAIN_ROWS:
        continue

    # 커스텀 vectorizer를 파이프라인에 그대로 사용(중복 생성 금지)
    pipe = Pipeline([
        ("tfidf", vectorizer),
        ("reg",   RidgeCV(alphas=ALPHAS, cv=5, scoring="neg_mean_squared_error"))
    ])
    pipe.fit(df.loc[mask_train, "_text_all"], y.loc[mask_train])
    text_models[col] = pipe

    if mask_pred.sum() > 0:
        y_hat = pipe.predict(df.loc[mask_pred, "_text_all"]).astype(float)
        lo, hi = y.loc[mask_train].quantile([0.01, 0.99])
        y_hat = np.clip(y_hat, lo, hi)
        df.loc[mask_pred, final_col] = np.round(y_hat, 1)

        # 불리언 마스크로 바로 업데이트
        fill_mask = mask_pred & (df[src_col] == "missing")
        df.loc[fill_mask, src_col] = "text"

print("텍스트 기반 보정 후 남은 NA 비율:",
      df[[f"{c}_text_final" for c in style_cols]].isna().mean().round(3).to_dict())

# -------------------- 힌트 단어(계수) 추출 --------------------

def top_terms_from_pipeline(pipe, top_k=TOP_K_TERMS):
    vec = pipe.named_steps["tfidf"]
    reg = pipe.named_steps["reg"]
    vocab = vec.get_feature_names_out()
    weights = np.asarray(reg.coef_).ravel()
    dfw = pd.DataFrame({"term": vocab, "weight": weights})
    dfw_pos = dfw.nlargest(top_k, "weight").reset_index(drop=True)
    dfw_neg = dfw.nsmallest(top_k, "weight").reset_index(drop=True)
    return dfw_pos, dfw_neg

term_hints = {}
for col in style_cols:
    if col in text_models:
        pos, neg = top_terms_from_pipeline(text_models[col], top_k=TOP_K_TERMS)
        term_hints[col] = {"positive": pos, "negative": neg}

for col in style_cols:
    if col in term_hints:
        print(f"\n=== {col.upper()} ↑ (positive terms) ===")
        print(term_hints[col]["positive"].to_string(index=False))
        print(f"\n=== {col.upper()} ↓ (negative terms) ===")
        print(term_hints[col]["negative"].to_string(index=False))

텍스트 기반 보정 후 남은 NA 비율: {'style_body_text_final': 0.0, 'style_richness_text_final': 0.0, 'style_smoke_text_final': 0.0, 'style_sweetness_text_final': 0.0}

=== STYLE_BODY ↑ (positive terms) ===
           term   weight
     oak ginger 0.485731
          pecan 0.469719
       pastries 0.457714
      honeycomb 0.446571
      peardrops 0.444715
           cake 0.430521
         prunes 0.416530
 cinnamon lemon 0.407234
         coffee 0.394841
   honey toffee 0.360090
        leather 0.357418
           meat 0.338654
       hazelnut 0.334499
         sponge 0.328482
     shortbread 0.326214
    apple black 0.318182
    fruit honey 0.316933
   blackcurrant 0.315891
         sherry 0.314239
      cranberry 0.313243
    sponge cake 0.310960
            jam 0.309547
        coconut 0.309071
vanilla custard 0.303533
          melon 0.296837

=== STYLE_BODY ↓ (negative terms) ===
            term    weight
         vanilla -0.860965
   peach vanilla -0.636959
           peach -0.437288
biscuits va

In [ ]:
df.head()

,name,main_info,volume_alcohol,price,style_body,style_richness,style_smoke,style_sweetness,characters,food_paring,bottler,age,country,region,casktype,colouring,vintage,nose,palate,finish,reviews,url,whisky_type,volume,alcohol,volume(cl),alcohol(%),price(£),name_norm,bottler_norm,bottler_canon,is_independent,bottling_year,bottling_decade,age_years_text,age_years,vintage_maybe,cask_code,age_,age_int,is_NAS,vintage_clean,bottling_decade_clean,cask_group,style_body_l1,style_richness_l1,style_smoke_l1,style_sweetness_l1,style_body_knn,style_richness_knn,style_smoke_knn,style_sweetness_knn,style_body_final,style_body_source,style_richness_final,style_richness_source,style_smoke_final,style_smoke_source,style_sweetness_final,style_sweetness_source,all_sources_equal,_text_all,style_body_text_final,style_body_text_source,style_richness_text_final,style_richness_text_source,style_smoke_text_final,style_smoke_text_source,style_sweetness_text_final,style_sweetness_text_source
0,Blended Scotch TB-BSW6 Year Old Thompson Bros,blended scotch whisky,70cl / 46%,£34.25,3.0,3.0,0.0,3.0,"Figs, Sultana, Cinnamon, Nutmeg, Toffee, Coffee",NaN,NaN,None,scotland,unknown_scotland,not specified,False,<NA>,"Plum sauce, miso, golden sultanas, cinder toff...","Golden sultanas, raisin loaf, buttered Jamaica...","Soft warm spice; allspice, nutmeg and cinnamon...","Stunning stuff for the money, might well be my...",https://www.thewhiskyexchange.com/p/67983/blen...,blended,70cl,46%,70.0,46.0,34.25,blended scotch tb-bsw6 year old thompson bros,,thompson bros,1,<NA>,None,None,<NA>,<NA>,None,NaN,17.584416,1,<NA>,<NA>,unknown,3.0,3.0,0.0,3.0,0.0,0.0,1.0,0.0,3.0,original,3.0,original,0.0,original,3.0,original,True,figs sultana cinnamon nutmeg toffee coffee plu...,3.0,original,3.0,original,0.0,original,3.0,original
1,Johnnie Walker Blue Label,blended scotch whisky,70cl / 40%,£158,3.0,3.0,1.0,2.0,"Rose, Hazelnut, Honey, Smoke, Dark Chocolate, ...","Octopus, Monkfish, Fondue",NaN,None,scotland,unknown_scotland,not specified,True,<NA>,"Smooth, silky cereal, syrupy flapjacks, polish...",A silky caress. Unexpectedly strong spicy entr...,A perfectly-integrated slow fade of the palate...,Un elixir. || Excellent Whisky. Smooth on the ...,https://www.thewhiskyexchange.com/p/552/johnni...,blended,70cl,40%,70.0,40.0,158.00,johnnie walker blue label,,distillery bottling,0,<NA>,None,None,<NA>,<NA>,None,NaN,17.584416,1,<NA>,<NA>,unknown,3.0,3.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,original,3.0,original,1.0,original,2.0,original,True,rose hazelnut honey smoke dark chocolate tobac...,3.0,original,3.0,original,1.0,original,2.0,original
2,The Big Ben,blended scotch whisky,50cl / 40%,£23.50,NaN,NaN,NaN,NaN,no description,NaN,NaN,None,scotland,unknown_scotland,not specified,True,<NA>,"Medium weight, malty nose, with hints of orang...","Medium depth, hints of smokiness and sweetness...",Elegant finish.,Very pleased with the whiskey. Highly recommend,https://www.thewhiskyexchange.com/p/18213/the-...,blended,50cl,40%,50.0,40.0,23.50,the big ben,,distillery bottling,0,<NA>,None,None,<NA>,<NA>,None,NaN,17.584416,1,<NA>,<NA>,unknown,3.0,2.9,1.1,2.9,0.0,0.0,1.0,0.0,0.0,knn,0.0,knn,1.0,knn,0.0,knn,True,no description medium weight malty nose with h...,3.1,text,3.0,text,0.5,text,3.0,text
3,J&B Rare,blended scotch whisky,70cl / 40%,£25.25,NaN,NaN,NaN,NaN,no description,NaN,NaN,None,scotland,unknown_scotland,not specified,True,<NA>,no description,no description,no description,Very nice || Was delighted to find this whisky...,https://www.thewhiskyexchange.com/p/544/jb-rare,blended,70cl,40%,70.0,40.0,25.25,j&b rare,,distillery bottling,0,<NA>,None,None,<NA>,<NA>,None,NaN,17.584416,1,<NA>,<NA>,unknown,3.0,2.9,1.1,2.9,0.0,0.0,1.0,0.0,0.0,knn,0.0,knn,1.0,knn,0.0,knn,True,no description no description no description n...,2.8,text,2.7,text,0.2,text,2.8,text
4,Dimple 15 Year OldLitre,blended scotch whisky,100cl / 43%,£69.95,3.0,3.0,0.0,3.0,"Caramel, Cinnamon, Vanilla, Dark Chocolate",NaN,NaN,15 year old,scotland,u

In [ ]:

style_cols = ["style_body", "style_richness", "style_smoke", "style_sweetness"]

# 최종 전략: 'text_only' | 'priority_text' | 'blend_mean'
STRATEGY = "priority_text"
AGREE_TOL = 0.5  # text vs knn 차이가 이하면 평균, 초과면 텍스트 선택 (priority_text에서만 사용)

def s(frame, col):
    if col not in frame.columns:
        # 없는 경우 NaN 시리즈 반환
        return pd.Series(np.nan, index=frame.index)
    x = frame[col]
    return pd.to_numeric(x, errors="coerce")

for c in style_cols:
    s_orig  = s(df, c)                   # 원본
    s_s1    = s(df, f"{c}_s1")           # 평균/중앙 1차
    s_knn   = s(df, f"{c}_knn")          # KNN
    s_text  = s(df, f"{c}_text_final")   # 텍스트

    if STRATEGY == "text_only":
        last = s_orig.copy()
        use_text = s_orig.isna() & s_text.notna()
        last[use_text] = s_text[use_text]

        source = np.where(s_orig.notna(), "original",
                 np.where(s_text.notna(), "text", "missing"))

    elif STRATEGY == "priority_text":
        # 우선순위: original > text > knn > stage1
        # 단, text & knn 모두 있으면: |text - knn| <= AGREE_TOL 이면 평균, 아니면 text
        agree = s_text.notna() & s_knn.notna() & (s_text.sub(s_knn).abs() <= AGREE_TOL)
        choose_text = s_text.notna() & ~agree
        blend_mean = agree

        last = s_orig.copy()
        m = s_orig.isna()

        # 합의(blend)
        last[m & blend_mean] = (s_text[m & blend_mean] + s_knn[m & blend_mean]) / 2
        # 텍스트 단독
        last[m & choose_text] = s_text[m & choose_text]
        # 텍스트 없음 → knn
        fill_knn = m & last.isna() & s_knn.notna()
        last[fill_knn] = s_knn[fill_knn]
        # 그래도 비면 stage1
        fill_s1 = m & last.isna() & s_s1.notna()
        last[fill_s1] = s_s1[fill_s1]

        source = np.full(len(df), "missing", dtype=object)
        source[s_orig.notna()] = "original"
        source[(s_orig.isna()) & blend_mean] = "text+knn_mean"
        source[(s_orig.isna()) & choose_text] = "text"
        source[(s_orig.isna()) & ~choose_text & ~blend_mean & s_knn.notna()] = "knn"
        source[(s_orig.isna()) & (pd.Series(source) == "missing") & s_s1.notna()] = "stage1"
        source = pd.Series(source).fillna("missing")

    elif STRATEGY == "blend_mean":
        # original > (mean(text, knn)) > text > knn > stage1
        both = s_text.notna() & s_knn.notna()
        blended = (s_text + s_knn) / 2

        last = s_orig.copy()
        m = s_orig.isna()
        last[m & both] = blended[m & both]
        fill_text = m & last.isna() & s_text.notna()
        last[fill_text] = s_text[fill_text]
        fill_knn  = m & last.isna() & s_knn.notna()
        last[fill_knn] = s_knn[fill_knn]
        fill_s1   = m & last.isna() & s_s1.notna()
        last[fill_s1] = s_s1[fill_s1]

        source = np.where(s_orig.notna(), "original",
                 np.where(both, "mean(text,knn)",
                 np.where(s_text.notna(), "text",
                 np.where(s_knn.notna(), "knn",
                 np.where(s_s1.notna(), "stage1", "missing")))))

    # 기록: *_last, *_last_source
    df[f"{c}_last"] = np.round(last, 1)
    df[f"{c}_last_source"] = source

# 점검
print("Final NA rate (last):",
      df[[f"{c}_last" for c in style_cols]].isna().mean().round(3).to_dict())
for c in style_cols:
    print(c, "->", df[f"{c}_last_source"].value_counts(dropna=False).to_dict())

Final NA rate (last): {'style_body_last': 0.0, 'style_richness_last': 0.0, 'style_smoke_last': 0.0, 'style_sweetness_last': 0.0}
style_body -> {'original': 618, 'text': 482}
style_richness -> {'original': 618, 'text': 482}
style_smoke -> {'original': 618, 'text+knn_mean': 377, 'text': 105}
style_sweetness -> {'original': 618, 'text': 482}


In [ ]:
df.head()

,name,main_info,volume_alcohol,price,style_body,style_richness,style_smoke,style_sweetness,characters,food_paring,bottler,age,country,region,casktype,colouring,vintage,nose,palate,finish,reviews,url,whisky_type,volume,alcohol,volume(cl),alcohol(%),price(£),name_norm,bottler_norm,bottler_canon,is_independent,bottling_year,bottling_decade,age_years_text,age_years,vintage_maybe,cask_code,age_,age_int,is_NAS,vintage_clean,bottling_decade_clean,cask_group,style_body_l1,style_richness_l1,style_smoke_l1,style_sweetness_l1,style_body_knn,style_richness_knn,style_smoke_knn,style_sweetness_knn,style_body_final,style_body_source,style_richness_final,style_richness_source,style_smoke_final,style_smoke_source,style_sweetness_final,style_sweetness_source,all_sources_equal,_text_all,style_body_text_final,style_body_text_source,style_richness_text_final,style_richness_text_source,style_smoke_text_final,style_smoke_text_source,style_sweetness_text_final,style_sweetness_text_source,style_body_last,style_body_last_source,style_richness_last,style_richness_last_source,style_smoke_last,style_smoke_last_source,style_sweetness_last,style_sweetness_last_source
0,Blended Scotch TB-BSW6 Year Old Thompson Bros,blended scotch whisky,70cl / 46%,£34.25,3.0,3.0,0.0,3.0,"Figs, Sultana, Cinnamon, Nutmeg, Toffee, Coffee",NaN,NaN,None,scotland,unknown_scotland,not specified,False,<NA>,"Plum sauce, miso, golden sultanas, cinder toff...","Golden sultanas, raisin loaf, buttered Jamaica...","Soft warm spice; allspice, nutmeg and cinnamon...","Stunning stuff for the money, might well be my...",https://www.thewhiskyexchange.com/p/67983/blen...,blended,70cl,46%,70.0,46.0,34.25,blended scotch tb-bsw6 year old thompson bros,,thompson bros,1,<NA>,None,None,<NA>,<NA>,None,NaN,17.584416,1,<NA>,<NA>,unknown,3.0,3.0,0.0,3.0,0.0,0.0,1.0,0.0,3.0,original,3.0,original,0.0,original,3.0,original,True,figs sultana cinnamon nutmeg toffee coffee plu...,3.0,original,3.0,original,0.0,original,3.0,original,3.0,original,3.0,original,0.0,original,3.0,original
1,Johnnie Walker Blue Label,blended scotch whisky,70cl / 40%,£158,3.0,3.0,1.0,2.0,"Rose, Hazelnut, Honey, Smoke, Dark Chocolate, ...","Octopus, Monkfish, Fondue",NaN,None,scotland,unknown_scotland,not specified,True,<NA>,"Smooth, silky cereal, syrupy flapjacks, polish...",A silky caress. Unexpectedly strong spicy entr...,A perfectly-integrated slow fade of the palate...,Un elixir. || Excellent Whisky. Smooth on the ...,https://www.thewhiskyexchange.com/p/552/johnni...,blended,70cl,40%,70.0,40.0,158.00,johnnie walker blue label,,distillery bottling,0,<NA>,None,None,<NA>,<NA>,None,NaN,17.584416,1,<NA>,<NA>,unknown,3.0,3.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,original,3.0,original,1.0,original,2.0,original,True,rose hazelnut honey smoke dark chocolate tobac...,3.0,original,3.0,original,1.0,original,2.0,original,3.0,original,3.0,original,1.0,original,2.0,original
2,The Big Ben,blended scotch whisky,50cl / 40%,£23.50,NaN,NaN,NaN,NaN,no description,NaN,NaN,None,scotland,unknown_scotland,not specified,True,<NA>,"Medium weight, malty nose, with hints of orang...","Medium depth, hints of smokiness and sweetness...",Elegant finish.,Very pleased with the whiskey. Highly recommend,https://www.thewhiskyexchange.com/p/18213/the-...,blended,50cl,40%,50.0,40.0,23.50,the big ben,,distillery bottling,0,<NA>,None,None,<NA>,<NA>,None,NaN,17.584416,1,<NA>,<NA>,unknown,3.0,2.9,1.1,2.9,0.0,0.0,1.0,0.0,0.0,knn,0.0,knn,1.0,knn,0.0,knn,True,no description medium weight malty nose with h...,3.1,text,3.0,text,0.5,text,3.0,text,3.1,text,3.0,text,0.8,text+knn_mean,3.0,text
3,J&B Rare,blended scotch whisky,70cl / 40%,£25.25,NaN,NaN,NaN,NaN,no description,NaN,NaN,None,scotland,unknown_scotland,not specified,True,<NA>,no description,no description,no description,Very nice || Was delighted to find this whisky...,https://www.thewhiskyexchange.com/p/544/jb-rare,blended,70cl,40%,70.0,40.0,25.25,j&b rare,,distillery bottling,0,<NA>,None,None,<NA>,<NA>,None,NaN,17.584416,1,<NA>,<

### 2. food_paring


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 78 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   name                         1100 non-null   object 
 1   main_info                    1100 non-null   object 
 2   volume_alcohol               1100 non-null   object 
 3   price                        1100 non-null   object 
 4   style_body                   618 non-null    float64
 5   style_richness               618 non-null    float64
 6   style_smoke                  618 non-null    float64
 7   style_sweetness              618 non-null    float64
 8   characters                   1100 non-null   object 
 9   food_paring                  30 non-null     object 
 10  bottler                      505 non-null    object 
 11  age                          503 non-null    object 
 12  country                      1100 non-null   object 
 13  region            

In [ ]:
df.sample(5)

,name,main_info,volume_alcohol,price,style_body,style_richness,style_smoke,style_sweetness,characters,food_paring,bottler,age,country,region,casktype,colouring,vintage,nose,palate,finish,reviews,url,whisky_type,volume,alcohol,volume(cl),alcohol(%),price(£),name_norm,bottler_norm,bottler_canon,is_independent,bottling_year,bottling_decade,age_years_text,age_years,vintage_maybe,cask_code,age_,age_int,is_NAS,vintage_clean,bottling_decade_clean,cask_group,style_body_l1,style_richness_l1,style_smoke_l1,style_sweetness_l1,style_body_knn,style_richness_knn,style_smoke_knn,style_sweetness_knn,style_body_final,style_body_source,style_richness_final,style_richness_source,style_smoke_final,style_smoke_source,style_sweetness_final,style_sweetness_source,all_sources_equal,_text_all,style_body_text_final,style_body_text_source,style_richness_text_final,style_richness_text_source,style_smoke_text_final,style_smoke_text_source,style_sweetness_text_final,style_sweetness_text_source,style_body_last,style_body_last_source,style_richness_last,style_richness_last_source,style_smoke_last,style_smoke_last_source,style_sweetness_last,style_sweetness_last_source
284,Gilbey's Spey RoyalBot.1950s,blended scotch whisky,75cl / 40%,£450,NaN,NaN,NaN,NaN,no description,NaN,NaN,None,scotland,unknown_scotland,not specified,Unknown,<NA>,no description,no description,no description,NaN,https://www.thewhiskyexchange.com/p/44863/gilb...,blended,75cl,40%,75.0,40.0,450.00,gilbey's spey royalbot.1950s,,distillery bottling,0,<NA>,1950s,None,<NA>,<NA>,None,NaN,17.584416,1,<NA>,1950,unknown,3.0,2.9,1.1,2.9,0.0,0.0,1.0,0.0,0.0,knn,0.0,knn,1.0,knn,0.0,knn,True,no description no description no description n...,3.1,text,3.0,text,0.6,text,3.0,text,3.1,text,3.0,text,0.8,text+knn_mean,3.0,text
197,Royal Salute Platinum JubileeRichmond Brooch (...,blended scotch whiskychivas brothers,70cl / 50.8%,"£14,775",3.0,3.0,0.0,3.0,"Orange, Dried Fruit, Clove, Caramel, Ginger, A...",NaN,chivas brothers,None,scotland,unknown_scotland,not specified,Unknown,<NA>,no description,no description,no description,NaN,https://www.thewhiskyexchange.com/p/66994/roya...,blended,70cl,50.8%,70.0,50.8,14775.00,royal salute platinum jubileerichmond brooch (...,chivas brothers,chivas brothers,0,<NA>,None,None,<NA>,<NA>,None,NaN,17.584416,1,<NA>,<NA>,unknown,3.0,3.0,0.0,3.0,0.0,0.0,1.0,0.0,3.0,original,3.0,original,0.0,original,3.0,original,True,orange dried fruit clove caramel ginger almond...,3.0,original,3.0,original,0.0,original,3.0,original,3.0,original,3.0,original,0.0,original,3.0,original
381,New York Ragtime Rye Bottled in Bond,american straight rye whiskey,70cl / 50%,£45.95,2.0,1.0,0.0,3.0,"White Pepper, Nutmeg, Cinnamon, Vanilla, Oak, ...",NaN,NaN,None,usa,new york,not specified,Unknown,<NA>,no description,no description,no description,Hard to believe this is only 4 y/o. The only p...,https://www.thewhiskyexchange.com/p/53694/new-...,rye,70cl,50%,70.0,50.0,45.95,new york ragtime rye bottled in bond,,distillery bottling,0,<NA>,None,None,<NA>,<NA>,None,NaN,10.000000,1,<NA>,<NA>,unknown,2.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,2.0,original,1.0,original,0.0,original,3.0,original,True,white pepper nutmeg cinnamon vanilla oak black...,2.0,original,1.0,original,0.0,original,3.0,original,2.0,original,1.0,original,0.0,original,3.0,original
746,Yamazaki 18 Year OldGift Box,japanese single malt whisky,70cl / 43%,£600,4.0,4.0,2.0,3.0,"Dark Chocolate, Nutmeg, Smoke, Ginger, Stewed ...",NaN,NaN,18 year old,japan,unknown_japan,not specified,Unknown,<NA>,no description,no description,no description,NaN,https://www.thewhiskyexchange.com/p/81705/yama...,single malt,70cl,43%,70.0,43.0,600.00,yamazaki 18 year oldgift box,,distillery bottling,0,<NA>,None,18 year old,18,<NA>,None,18.0,18.000000,0,<NA>,<NA>,unknown,4.0,4.0,2.0,3.0,0.0,0.0,1.0,0.0,4.0,original,4.0,original,2.0,original,3.0,original,True,dark chocolate nutmeg smoke ginger stewed frui...,4.0,original,4.0,original,2.0,original,3.0,original,4.0,original,4.0,original,

In [ ]:
df.columns

Index(['name', 'main_info', 'volume_alcohol', 'price', 'style_body',
       'style_richness', 'style_smoke', 'style_sweetness', 'characters',
       'food_paring', 'bottler', 'age', 'country', 'region', 'casktype',
       'colouring', 'vintage', 'nose', 'palate', 'finish', 'reviews', 'url',
       'whisky_type', 'volume', 'alcohol', 'volume(cl)', 'alcohol(%)',
       'price(£)', 'name_norm', 'bottler_norm', 'bottler_canon',
       'is_independent', 'bottling_year', 'bottling_decade', 'age_years_text',
       'age_years', 'vintage_maybe', 'cask_code', 'age_', 'age_int', 'is_NAS',
       'vintage_clean', 'bottling_decade_clean', 'cask_group', 'style_body_l1',
       'style_richness_l1', 'style_smoke_l1', 'style_sweetness_l1',
       'style_body_knn', 'style_richness_knn', 'style_smoke_knn',
       'style_sweetness_knn', 'style_body_final', 'style_body_source',
       'style_richness_final', 'style_richness_source', 'style_smoke_final',
       'style_smoke_source', 'style_sweetness_final'

In [ ]:
last_cols = ['name', 'main_info','whisky_type','volume(cl)', 'alcohol(%)', 'price(£)',
             'style_body_last', 'style_richness_last',
              'style_smoke_last', 'style_sweetness_last',
             'characters',
             'food_paring',
             'bottler', 'bottling_decade_clean','bottler_canon', 'is_independent','bottling_year',
             'age','age_','age_int','is_NAS',
             'country', 'region',
             'casktype', 'cask_group',
            'colouring',
             'vintage','vintage_clean',
             'nose', 'palate', 'finish', 'reviews', '_text_all',
]
df[last_cols].head()
df_processed = df[last_cols]
df_processed.to_csv('/content/drive/MyDrive/week_hands_on_project5/whiskey/whisky_processed2.csv', index=False)

In [ ]:
missing_rate = (
    df.isnull()
      .sum()
      .reset_index()
      .rename(columns={'index': 'feature', 0: 'missing_count'})
)
missing_rate["missing_rate(%)"] = (missing_rate["missing_count"] / len(df)) * 100
missing_rate.sort_values("missing_rate(%)", ascending=False, inplace=True)

In [ ]:
display(missing_rate)

,feature,missing_count,missing_rate(%)
36,vintage_maybe,1092,99.272727
16,vintage,1092,99.272727
41,vintage_clean,1092,99.272727
32,bottling_year,1073,97.545455
9,food_paring,1070,97.272727
...,...,...,...
73,style_richness_last_source,0,0.000000
74,style_smoke_last,0,0.000000
75,style_smoke_last_source,0,0.000000
76,style_sweetness_last,0,0.000000


In [ ]:
overall_missing_rate = df.isnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100
print(f"전체 평균 결측률: {overall_missing_rate:.2f}%")

전체 평균 결측률: 16.17%
